# TabNet

In [144]:
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

import pandas as pd
import numpy as np
np.random.seed(0)

import os
from pathlib import Path

from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
def get_data():
    PATH = Path('../data/houseprice/')
    df_train = pd.read_csv(PATH/'train.csv', sep=',', header=0)
    df_y = df_train['SalePrice']
    df_train.drop(['SalePrice'], axis=1, inplace=True)
    df_test = pd.read_csv(PATH/'test.csv', sep=',', header=0)

    cat_names = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir',
        'Electrical', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',
        'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
        'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
        'MoSold', 'YrSold', 'SaleType', 'SaleCondition']

    cont_names = [name for name in df_train.columns if name not in cat_names]
    proc_train, proc_test = proc_df(cont_names, cat_names, df_train, df_test)

    x_train = torch.tensor(proc_train.values[:1168], dtype=torch.float32)
    y_train = torch.tensor(df_y.values[:1168], dtype=torch.float32) #.reshape(-1,1)
    x_valid = torch.tensor(proc_train.values[1168:], dtype=torch.float32)
    y_valid = torch.tensor(df_y.values[1168:], dtype=torch.float32) #.reshape(-1,1)
    x_test = torch.tensor(proc_test.values, dtype=torch.float32)

    return (x_train, y_train, x_valid, y_valid, x_test)

In [6]:
PATH = Path('../data/houseprice/')

In [206]:
train = pd.read_csv(PATH/'train.csv', sep=',', header=0)
train.drop(['Id'], axis=1, inplace=True)

target = 'SalePrice'

if "Set" not in train.columns:
    train["Set"] = np.random.choice(['train', 'valid', 'test'], p=[.8, .1, .1], size=(train.shape[0], ))
train_indices = train[train.Set=='train'].index
valid_indices = train[train.Set=='valid'].index
test_indices = train[train.Set=='test'].index

In [207]:
types = train.dtypes
nunique = train.nunique()
categorical_columns = []
categorical_dims = {}

In [208]:
for col in train.columns:
    if train[col].dtype == 'object':
        if train[col].hasnans:
            train[col].fillna('VV_likely', inplace=True)
    elif train[col].dtype != 'object' and train[col].nunique() < 200:
        print('no object and < 200')
        if train[col].hasnans:
            train[col].fillna(-1., inplace=True)
    else:
        print('no object and >= 200')
#         if train[col].hasnans:
#             train[col].fillna()

no object and < 200
no object and < 200
no object and >= 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and >= 200
no object and >= 200
no object and < 200
no object and >= 200
no object and >= 200
no object and >= 200
no object and >= 200
no object and < 200
no object and >= 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and >= 200
no object and >= 200
no object and >= 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and < 200
no object and >= 200


In [202]:
train[col].unique()

array(['VV_likely', 'Grvl', 'Pave'], dtype=object)

In [201]:
col = 'Alley'

In [193]:
train[col].hasnans

True

In [188]:
train.Alley.unique()

array(['VV_likely', 'Grvl', 'Pave'], dtype=object)

In [187]:
train[col].fillna('VV_likely', inplace=True)

In [182]:
train['Alley'].fillna('VV_likely', inplace=True)

In [106]:
l_enc = LabelEncoder()

In [107]:
train['Alley'] = l_enc.fit_transform(train[col].values)

In [112]:
train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd',
       'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond',
       'PavedDrive', 'Wo

In [123]:
train[col].unique()

array([10520.047120418849, 'Grvl', 'Pave'], dtype=object)

In [134]:
if types[col] == 'object' or nunique[col] < 200:
    print(f'obj: {col}:\t{train[col].nunique()}')    
    if train[col].dtype == 'object':
        train[col].fillna('VV_likely', inplace=True)
    else:
        train[col].fillna(-1., inplace=True)
    l_enc = LabelEncoder()
    train[col] = l_enc.fit_transform(train[col].values)

obj: MasVnrType:	4


In [128]:
train[col].unique()

array([2, 0, 1])

In [143]:
train['Alley'].unique()

array([nan, 'Grvl', 'Pave'], dtype=object)

In [147]:
col

'Alley'

In [159]:
train[col].unique()

array([nan, 'Grvl', 'Pave'], dtype=object)

In [137]:
for col in train.columns:
    if types[col] == 'object' or nunique[col] < 200:
        print(f'obj: {col}:\t{train[col].nunique()}')
        s_imp = SimpleImputer()
        if train[col].dtype == 'object':
            s_imp.fit_transform()
            train[col].fillna("VV_likely", inplace=True)
        else:
            train[col].fillna(-1., inplace=True)
        l_enc = LabelEncoder()        
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        print(f'num: {col}')
        train.fillna(train.loc[train_indices, col].mean(), inplace=True)

obj: MSSubClass:	15
obj: MSZoning:	5
obj: LotFrontage:	110
num: LotArea
obj: Street:	2
obj: Alley:	3


TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']